In [ ]:
import statistics as st

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df = pd.read_excel("/cons_us.xls")

In [ ]:
df

,obs,CE,PDI
0,1985Q1,72.823759,0.817916
1,1985Q2,75.695825,0.849901
2,1985Q3,80.031250,0.891667
3,1985Q4,81.237113,0.888660
4,1986Q1,79.608826,0.883651
5,1986Q2,83.418891,0.907598
6,1986Q3,86.712329,0.937829
7,1986Q4,86.281250,0.937500
8,1987Q1,86.509927,0.949843
9,1987Q2,88.417191,0.966457


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   obs     38 non-null     object 
 1   CE      38 non-null     float64
 2   PDI     38 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.0+ KB


In [ ]:
df.set_index('obs',inplace=True)

In [ ]:
df['CE-1'] = df['CE'].shift(+1)

## α) Εκτίμηση Μοντέλου Μερικής Προσαρμογής

In [ ]:
 import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df.dropna(inplace=True)

In [ ]:
Y = df['CE']

In [ ]:
X = df[['CE-1','PDI']]

In [ ]:
X = sm.add_constant(X)

In [ ]:
model = sm.OLS(Y,X)

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     CE   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     231.1
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           1.62e-20
Time:                        20:58:51   Log-Likelihood:                -89.262
No. Observations:                  37   AIC:                             184.5
Df Residuals:                      34   BIC:                             189.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.7420      9.338     -1.150      0.2

# β) Ερμηνεία των εκτιμόμενων συντελεστών
# γ) Υπολογισμός έμμεσου συντελεστή προσαρμογής

In [ ]:
results.params

const   -10.741967
CE-1      0.849750
PDI      27.186180
dtype: float64

In [ ]:
results.params['CE-1']

0.8497495917738617

In [ ]:
Lambda = 1 - results.params['CE-1']

In [ ]:
Lambda

0.15025040822613833

In [ ]:
B1 = results.params['const'] / Lambda

In [ ]:
B1

-71.49376514630963

In [ ]:
B2 = results.params['PDI'] / Lambda

In [ ]:
B2

180.939139681254

In [ ]:
time = 1 / Lambda

In [ ]:
time

6.6555559602535235



Ο συντελεστής προσαρμογής είναι ο Lambda = 0.15025040822613833 και ο χρόνος προσαρμογής είναι 1/Lambda = 6.6555559602535235. 
O συντελεστής της Μακροχρόνιας Αντίδραση του CE είναι  Β1 = -71.49376514630963 και η Βραχυχρόνια Αντίδραση του στη μοναδιαία αλλαγή του PDI είναι Β2λ = 27.1862

## δ) Έλεγχος h Durbin

In [ ]:
from statsmodels.stats.stattools import durbin_watson

In [ ]:
import math 

In [ ]:
durbin_watson(results.resid)

1.6229827263264076

In [ ]:
results.bse

const     9.338104
CE-1      0.044501
PDI      10.275342
dtype: float64

In [ ]:
df.shape

(37, 3)

In [ ]:
type(len(df))

int

In [ ]:
float(results.bse[1])

0.04450148609896932

In [ ]:
x = len(df)/(1-(len(df)*float(results.bse[1]**2)))

In [ ]:
h = (1-durbin_watson(results.resid)/2)*math.sqrt(x)

In [ ]:
h

1.1911226272363282

h-Durbin = 1.1911226272363282
To h-Durbin δεν είναι μεγαλύτερο του Ζο.ο5 = 1.96, άρα δέχομαι την H0 και δεν υπάρχει αυτοσυσχέτιση.

δ) Ο έλεγχος LM

In [76]:
from tabulate import tabulate

In [78]:
for i in range(1,5):
  print("\nnlags = ",i)
  print(tabulate([['lm_stat',sm.stats.diagnostic.acorr_breusch_godfrey(results=results,nlags=i,store=False)[0]],["lmpval",sm.stats.diagnostic.acorr_breusch_godfrey(results=results,nlags=i,store=False)[1]],["fval",sm.stats.diagnostic.acorr_breusch_godfrey(results=results,nlags=i,store=False)[2]],["fpval",sm.stats.diagnostic.acorr_breusch_godfrey(results=results,nlags=i,store=False)[3]]], headers=["Results", "Values"]))


nlags =  1
Results      Values
---------  --------
lm_stat    0.996599
lmpval     0.318135
fval       0.913463
fpval      0.346147

nlags =  2
Results      Values
---------  --------
lm_stat    3.87245
lmpval     0.144247
fval       1.87032
fpval      0.17053

nlags =  3
Results      Values
---------  --------
lm_stat    5.65548
lmpval     0.129629
fval       1.86444
fpval      0.156182

nlags =  4
Results         Values
---------  -----------
lm_stat    16.2142
lmpval      0.00274475
fval        5.85049
fpval       0.00132663
